In [1]:
import time
!pip install ultralytics

In [2]:
!pip install -U ipywidgets

^C
ERROR: Operation cancelled by user


In [1]:
from ultralytics import YOLO

### Training the model

In [ ]:
results = model.train(
    data='/kaggle/input/80-10-10-dataset/data.yaml',
    epochs=100,
    imgsz=640,
    patience=50,
    device=0,
    batch=64,
    optimizer='SGD',
    momentum=0.95
)

### Evaluation

In [ ]:
model = YOLO("best_80_10_10.pt")

metrics = model.val()
metrics.box.map
metrics.box.map50
metrics.box.map75
metrics.box.maps

### hyperparameter-tuning

- in our case SGD as optimizer is better that AdamW


In [ ]:
model = YOLO("best_s_version.pt")

- lr0 = [0.0005, **0.001**, 0.005]
- lrf = [0.01, **0.005**]
- batch = [16, **32**, 64]
- momentum = [0.85, **0.9**, 0.95]
- warmup_momentum = [**0.5**]
- warmup_bias_lr = [**0.05**, 0.1, 0.2]
- weight_decay = [0.0001, **0.0005**, 0.001]
- patience = [**30**, 50]
- imgsz = [**640**, 800, 1024]


In [ ]:
results = model.train(
    data='/kaggle/input/80-10-10-dataset/data.yaml',
    epochs=50,
    imgsz=640,
    patience=30,
    device=0,
    batch=32,
    optimizer='SGD',
    lr0=0.001,
    lrf=0.005,
    momentum=0.9,
    warmup_momentum=0.5,
    warmup_bias_lr=0.05,
    weight_decay=0.0005,
)


### Testing the model

In [1]:
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator
import cv2

In [ ]:
model = YOLO("best_s_version.pt")
video_path = 'sample_02.mp4'
cap = cv2.VideoCapture(video_path)

In [8]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)

    for r in results:
        annotator = Annotator(frame)

        boxes = r.boxes
        for box in boxes:
            b = box.xyxy[0]
            c = box.cls
            annotator.box_label(b, model.names[int(c)])

    frame = annotator.result()
    cv2.imshow("Object Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

### format the datetime for database purpose 

In [3]:
from datetime import datetime

def format_datetime(dt: datetime):
    formatted_datetime = dt.strftime('%Y-%m-%d %H:%M:%S')
    return formatted_datetime


### Alarm Testing

In [ ]:
!pip install pydub pyaudio

In [ ]:
from alarm import AlarmDetector
from pydub import AudioSegment
from pydub.playback import play

In [ ]:
import time

model = YOLO("../models/best_s_version.pt")
video_path = '../video_samples/sample_01.mp4'
alarm_sound = AudioSegment.from_file('alarm.wav')
cap = cv2.VideoCapture(video_path)
calc = AlarmDetector(model, 300, 50)
alarm = False

while True:
    ret, frame = cap.read(10)
    if not ret:
        break

    results = model(frame, stream=True)

    for r in results:
        time.sleep(.01)
        annotator = Annotator(frame)

        boxes = r.boxes
        calc.add_boxes(boxes)
        for box in boxes:
            b = box.xyxy[0]
            c = box.cls
            annotator.box_label(b, model.names[int(c)])

        if calc.check_alarm() and not alarm:
            alarm = True
            play(alarm_sound)
            # add to database and send notifications

    frame = annotator.result()
    cv2.imshow("Object Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [6]:
cap.release()
cv2.destroyAllWindows()